In [6]:
from lusidtools.jupyter_tools import toggle_code

"""Futures Valuation Workflow

Attributes
----------
futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Computing Intraday P&L for Futures with Synthetic Cash

In this notebook, we demonstrate how P&L can be computed for Futures instruments under two different cash accounting treatments using the Synthetic Cash method. We look at these treatments over the course of three days for both a Dec 21 and Mar 22 Bund Futures contract. Note that we maintain the same market quotes and transaction amounts for each Futures contract to more easily demonstrate how the PV and P&L measures evolve over time.

### Synthetic Cash Method
The Synthetic Cash method of modelling Futures contracts introduces two key differences from the standard model:

1. The PVs of our Futures positions are defined as the market exposure of the position rather than the unrealized P&L of the position.
2. A synthetic cash position is created upon purchase or sale of a Futures contract such that the aggregate PV of the Future and cash position net to 0 for a given transaction.

### Dec 21 Bund Futures with Unrealized P&L (‘non-close out’)
In our first example, we look at a Futures contract whereby its daily P&L is left as unrealised on the position as opposed to marked-to-market on a daily basis.

### Mar 22 Bund Futures with Realized P&L ('close out’)
In our second example, we look at a Futures contract whereby the previous day's P&L is realized at the beginning of each day and booked down into a separate cash line item. We then reset the cost basis of our holding to reflect this mark-to-market such that the computed PV at the start of each day is 0.

## Table of Contents
* [1. Create Portfolio](#-Create-Portfolio)
* [2. Create Properties](#-Create-Properties)
* [3. Create Futures](#-Create-Futures)
* [4. Create Transactions](#-Transactions)
* [5. Quotes](#-Quotes)
* [6. Valuations](#-Valuations)

In [7]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Create API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    app_name="futuresLoader"
)

In [8]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [9]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"

# 1. Create Portfolio<a class="anchor" id="-Create-Portfolio"></a>

In order to support the grouping of our Futures contracts with their offsetting synthetic cash positions, we're going to use the LUSID subholding key feature. This allows us to tie two holdings together by one or more values specified within our input transactions.

In our example, we're going to create two subholding keys: FutCode and FutHoldType. 

- **FutCode** - This will serve as our 'top level' aggregation code and will allow us to tie our Futures contract positions to our synthetic and P&L cash positions. This will take on the value of either 'Dec21' or 'Mar22'.
- **FutHoldType** - This will serve as our 'second level' aggregation and will allow us to split out the underlying transactions that make up a single Futures aggregation. In our example, this will take on the value of 'PnLCash', 'SyntheticCash', 'Dec21Future', or 'Mar21Future'.

In [10]:
portfolio_code = "SynthCashFuturesPortWithDiffCostBasis"

try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            sub_holding_keys=[f"Transaction/{scope}/FutCode", f"Transaction/{scope}/FutHoldType"],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# 2. Create Properties<a class="anchor" id="-Create-Properties"></a>

In order to populate our subholding keys, we're going to create five transaction properties. We describe each property below:

1. **FutCode** - This property maps each transaction to a top level Futures aggregation. In our example, this will take on the value of 'Dec21' or 'Mar22'. This property maps directly to the corresponding portfolio subholding key.
2. **FutHoldType** - This property represents the 'second level' of our Futures valuation aggregation and is used to split out our aggregations into their component parts. In our example, possible values are: 'Dec21Future', 'Mar22Future', 'PnLCash', and 'SyntheticCash'. This property maps to our second subholding key and derives its value from the FutCodeSubType, FutSynthCashCode, and FutPnLCashCode transaction properties.
3. **FutCodeSubType** - This property is used to distinguish our Futures transactions from the cash transactions within the second level of aggregation. The value is composed of the FutCode property suffixed with 'Future'
4. **FutSynthCashCode** - Used to mark the Synthetic Cash component of a transaction.
5. **FutPnLCashCode** - Used to mark the PnL cash component of a transaction.

In [11]:
properties = [
    ("FutCode", "string"),
    ("FutHoldType", "string"),
    ("FutCodeSubType", "string"),    
    ("FutSynthCashCode", "string"),
    ("FutPnLCashCode", "string"),
]

for property_code, dtype in properties:
    try:
        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
            )
        )
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

# 3. Create Futures<a class="anchor" id="-Create-Futures"></a>

In [12]:
# Define function that creates futures
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

## 3.1 Create Bund Futures Contract Expiring Dec 21

In [13]:
start_date = datetime(2021, 3, 8, tzinfo=pytz.utc)
effectiveAt = datetime.today().replace(tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "Z"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Dec21"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = None
identifier = "FutBund001"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D62


## 3.2 Create Bund Futures Contract Expiring Mar 22

In [14]:
start_date = datetime(2021, 6, 8, tzinfo=pytz.utc)
effectiveAt = datetime.today().replace(tzinfo=pytz.utc)
maturity_date = datetime(2022, 3, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "H"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Mar22"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = 0
identifier = "FutBund002"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D63


# 4. Transactions<a class="anchor" id="-Transactions"></a>

## 4.1 Create Transaction Types

To book Synthetic Cash based Futures transactions, there's two key considerations to make. First, we need to create a set of transactions to faciliate the purchase of our Futures including the realization of P&L and the offsetting of PV via a synthetic cash holding. Second, we must map our various transaction properties to our two subholding keys: **FutCode** and **FutHoldType**. 

#### 4.1.2 Transaction Definitions

##### OpenContract
The first transaction type 'OpenContract' is a standard transaction used to enter into our Futures positions. This contains two movements. The first increases the size of the position based on a quantity provided, while the second creates an offsetting synthetic cash position.

##### RealizePnLIncrease and RealisePnLDecrease

The second two transaction types 'RealisePnLIncrease' and 'RealisePnLDecrease' are specific to the close out method used by our Mar 22 Futures position. These transactions exist to adjust the cost basis up or down depending on the position's daily P&L.

These transactions have three movements:

- The first movement updates the cost basis of the Futures position to reflect the daily mark-to-market change. This is done by adjusting the total_consideration of the position up or down while leaving the quantity of the position unchanged (in the transaction file we specify a quantity of 0). 
- The second movement records the cash generated from the daily mark-to-market change. This again gets its amount from the total_consideration value in the Excel file.
- Lastly, we create a synthetic cash amount to offset the realised P&L from the second movement.

#### 4.1.3 Subholding Key Mapping

Within each movement of our three transactions, we leverage LUSID's transaction mapping feature to populate the values of our subholding keys. Each mapping is based on whether the movement is related to the Future instrument itself, realised P&L cash, or synthetic cash.

In [15]:
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="OpenContract",
                description="An FuturesTxnType transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutCodeSubType")
                ]
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutSynthCashCode")
                ]
            )
        ],
        properties=None,
    ),
    models.TransactionConfigurationDataRequest(    
        aliases=[        
            models.TransactionConfigurationTypeAlias(
                type="RealisePnLIncrease",
                description="Increase cost basis and adjust cash",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutCodeSubType")
                ]
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutPnLCashCode")
                ]                
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutSynthCashCode")
                ]
            )
        ],
        properties=None,
    ),
    models.TransactionConfigurationDataRequest(    
        aliases=[        
            models.TransactionConfigurationTypeAlias(
                type="RealisePnLDecrease",
                description="Decrease cost basis and adjust cash",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutCodeSubType")
                ]
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutPnLCashCode")
                ]
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutCode", map_from=f"Transaction/{scope}/FutCode"),
                    models.TransactionPropertyMappingRequest(property_key=f"Transaction/{scope}/FutHoldType", map_from=f"Transaction/{scope}/FutSynthCashCode")
                ]
            )
        ],
        properties={}
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

## 4.2 Load Transaction Data

In [16]:
# Read in transaction data read from file
futures_transactions = pd.read_excel("data/synthetic_cash_futures_data.xlsx", sheet_name="transactions")

In [17]:
for index, row in futures_transactions.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=row['portfolio'],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["trade_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency_id"]
                ),
                properties={
                    f"Transaction/{scope}/FutCode": models.ModelProperty(
                        key=f"Transaction/{scope}/FutCode",
                        value=models.PropertyValue(label_value=row['fut_code']),
                    ),
                    f"Transaction/{scope}/FutCodeSubType": models.ModelProperty(
                        key=f"Transaction/{scope}/FutCodeSubType",
                        value=models.PropertyValue(label_value=row['fut_code'] + "Future"),
                    ),
                    f"Transaction/{scope}/FutSynthCashCode": models.ModelProperty(
                        key=f"Transaction/{scope}/FutSynthCashCode",
                        value=models.PropertyValue(label_value=row['fut_synth_cash_code']),
                    ),
                    f"Transaction/{scope}/FutPnLCashCode": models.ModelProperty(
                        key=f"Transaction/{scope}/FutPnLCashCode",
                        value=models.PropertyValue(label_value=row['fut_pnl_cash_code']),
                    ),
                },
            )
        ],
    )

# 5. Quotes<a class="anchor" id="-Quotes"></a>

## 5.1 Book Quotes

In [18]:
# Read in futures data from file
futures_prices = pd.read_excel("data/synthetic_cash_futures_data.xlsx", sheet_name="prices")
# Output futures data
futures_prices

,date,prices,luid,id_type,currency,scaling_factor,note
0,2021-09-01T08:00:00Z,100.00,FutBund001,ClientInternal,EUR,100,Day 1 opening price
1,2021-09-01T12:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 intraday MTM price
2,2021-09-01T17:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 closing price
3,2021-09-02T08:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 2 opening price
4,2021-09-02T14:15:00Z,99.50,FutBund001,ClientInternal,EUR,100,Day 2 intraday trade price
5,2021-09-02T17:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day 2 closing price
6,2021-09-03T08:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day 3 opening price
7,2021-09-03T14:15:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day 3 intraday trade price
8,2021-09-03T17:00:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day 3 closing price
9,2021-09-01T08:00:00Z,100.00,FutBund002,ClientInternal,EUR,100,Day 1 opening price


In [19]:
for index, row in futures_prices.iterrows():

    instrument_quotes = {
        "upsert_request_1": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["luid"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["date"],
            ),
            metric_value=models.MetricValue(value=row["prices"], unit=row["currency"]),
            scale_factor=row["scaling_factor"]            
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=quotes_scope, request_body=instrument_quotes
    )

# 6. Valuations<a class="anchor" id="-Valuations"></a>

## 6.1 Create valuation recipe

In [20]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="futuresValuation",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Future",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 6.2 Create daily valuation function

The get_daily_fut_val() function when given a date & time along with a portfolio returns a valuation of the held instruments with an indicator of their price relative to the agreed future price. This valuation therefore enables you to know if your futures are worth more or less than you agreed to pay for them at any given time.

In [21]:
def get_daily_fut_val(date, portfolio_code, aggregateTopLevel):

    # Define valuation aggregation and grouping parameters
    metricsList=[
            models.AggregateSpec(f"Transaction/{scope}/FutCode", "Value"),        
            models.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Holding/default/Units", "Value"),
            models.AggregateSpec("Holding/default/Cost", "Value"),
            models.AggregateSpec("Valuation/PV/Amount", "Sum"),
            models.AggregateSpec("Valuation/Exposure/Amount", "Sum"),
        ]
        
    if (aggregateTopLevel == False):
        metricsList.append(models.AggregateSpec(f"Transaction/{scope}/FutHoldType", "Value"))
    
    group_by = [f"Transaction/{scope}/FutCode"]
    
    if (aggregateTopLevel == False):
        group_by.append(f"Transaction/{scope}/FutHoldType")

        
    # Build and run valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code="futuresValuation"),
        metrics=metricsList,
        group_by=group_by,
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data
    
    vals_df = pd.DataFrame(val_data)
    
    sorted_df = vals_df
    
    if (aggregateTopLevel == True):
        sorted_df = vals_df.sort_values(by=[f"Transaction/{scope}/FutCode"], ascending=True)
    else:
        sorted_df = vals_df.sort_values(by=[f"Transaction/{scope}/FutHoldType"], ascending=True)

        
    # Rename result columns
    columnsToRename={
            "Instrument/Definition/ContractSize": "Contract Size",
            "Quotes/Price": "Price",
            "Holding/default/Units": "Units",
            "Holding/default/Cost": "Cost",
            "Sum(Valuation/PV/Amount)": "PV",
            "Sum(Valuation/Exposure/Amount)": "Exposure",
            f"Transaction/{scope}/FutCode": "FutCode",
    }   
        
    if (aggregateTopLevel == False):
        columnsToRename[f"Transaction/{scope}/FutHoldType"] = "FutHoldType"          

    sorted_df.rename(
        columns=columnsToRename,
        inplace=True,
    )
    
    
    # Re-order result columns
    col_list = ['FutCode']
    
    if (aggregateTopLevel == False):
        col_list.append("FutHoldType")
    
    col_list.extend(['PV', 'Exposure'])
    
    col_list.extend(['Contract Size', 'Price', 'Units', 'Cost'])

    sorted_df = sorted_df.reindex(columns=col_list)

    return sorted_df

## 6.3 Daily valuations

## Day 1

#### Start of Day

At 8:00am on Day 1, we enter into our two Bund Futures contracts at a price of 100.00.

In [22]:
futures_transactions.iloc[0:2]

,txn_id,fut_code,fut_synth_cash_code,fut_pnl_cash_code,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
0,futa_txn_001,Dec21,SynthCash,None,OpenContract,2021-09-01T08:00:00Z,10,FutBund001,EUR,100.00,1000000,SynthCashFuturesPortWithDiffCostBasis
1,futb_txn_001,Mar22,SynthCash,None,OpenContract,2021-09-01T08:00:00Z,10,FutBund002,EUR,100.00,1000000,SynthCashFuturesPortWithDiffCostBasis


We can first see a disaggregated view of our positions where we have two Futures positions with a PV of 1,000,000 EUR as well as two offsetting synthetic cash positions with PVs of -1,000,000 EUR.

In [23]:
get_daily_fut_val("2021-09-01T08:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,000,000.00","1,000,000.00","100,000.00",100.00,10.00,"1,000,000.00"
3,Mar22,Mar22Future,"1,000,000.00","1,000,000.00","100,000.00",100.00,10.00,"1,000,000.00"
1,Dec21,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"
2,Mar22,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"


We we then role these up based on the 'FutCode' subholding key, we can see the aggregate PV is 0.

In [24]:
get_daily_fut_val("2021-09-01T08:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,0.00,0.00,None,100.00,None,None
1,Mar22,0.00,0.00,None,100.00,None,None


#### Intraday

At 12:00pm noon, we decide to run an intraday mark-to-market valuation. We can see that the contract price for both instruments has moved from the original purchase price of 100.00, to 100.50. As a result, we see the PV of the two Futures positions has increased by 5,000 EUR while the synthetic cash position amounts remain unchanged.

In [25]:
get_daily_fut_val("2021-09-01T12:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,000,000.00"
3,Mar22,Mar22Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,000,000.00"
1,Dec21,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"
2,Mar22,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"


When we look at the aggregated view, we can see the PV of the Futures holdings have increased by 5,000 EUR.

In [26]:
get_daily_fut_val("2021-09-01T12:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"5,000.00","5,000.00",None,100.50,None,None
1,Mar22,"5,000.00","5,000.00",None,100.50,None,None


#### End of Day

At the end of the day, we see the contract prices remain at 100.50 yielding us the same valuation we produced intraday

In [27]:
get_daily_fut_val("2021-09-01T17:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,000,000.00"
3,Mar22,Mar22Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,000,000.00"
1,Dec21,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"
2,Mar22,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"


In [28]:
get_daily_fut_val("2021-09-01T17:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"5,000.00","5,000.00",None,100.50,None,None
1,Mar22,"5,000.00","5,000.00",None,100.50,None,None


## Day 2
#### Start of Day

At 8:00am on Day 2, we book a 'RealisePnlIncrease' transaction in order to adjust the cost basis of our Mar 22 Bund Future upwards such that its aggregated start of day PV is 0. This adjustment can be automatically applied in LUSID removing the need to manually book these transactions each day.

In [29]:
futures_transactions.iloc[2:3]

,txn_id,fut_code,fut_synth_cash_code,fut_pnl_cash_code,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
2,rpnlb_txn_001,Mar22,SynthCash,PnLCash,RealisePnLIncrease,2021-09-02T08:00:00Z,0,FutBund002,EUR,0.00,5000,SynthCashFuturesPortWithDiffCostBasis


We again run a start of day valuation immediately afterwards and can see that both the Dec 21 and Mar 22 Futures contracts have a PV of 1,005,000 EUR. We note however that a new cash entry with a FutHoldType subholding key value of 'PnLCash' appears. This amount is the realized P&L from the previous day's end of day mark-to-market gain (5,000 EUR). We also add an offsetting synthetic cash amount to the Mar 22 Future of the same amount. We can see the 'SynthCash' bucket has decreased from -1,000,000 EUR to -1,005,000 EUR.

In [30]:
get_daily_fut_val("2021-09-02T08:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,000,000.00"
4,Mar22,Mar22Future,"1,005,000.00","1,005,000.00","100,000.00",100.50,10.00,"1,005,000.00"
3,Mar22,PnLCash,"5,000.00","5,000.00",1.00,NaN,"5,000.00","5,000.00"
1,Dec21,SynthCash,"-1,000,000.00","-1,000,000.00",1.00,NaN,"-1,000,000.00","-1,000,000.00"
2,Mar22,SynthCash,"-1,005,000.00","-1,005,000.00",1.00,NaN,"-1,005,000.00","-1,005,000.00"


In [31]:
get_daily_fut_val("2021-09-02T08:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"5,000.00","5,000.00",None,100.50,None,None
1,Mar22,"5,000.00","5,000.00",None,100.50,None,None


#### Intraday


At 2:15pm, we decide to book a new transaction in the amount of 5 contracts of each Future at a price of 99.50.

In [32]:
futures_transactions.iloc[3:5]

,txn_id,fut_code,fut_synth_cash_code,fut_pnl_cash_code,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
3,futa_txn_002,Dec21,SynthCash,None,OpenContract,2021-09-02T14:15:00Z,5,FutBund001,EUR,99.50,497500,SynthCashFuturesPortWithDiffCostBasis
4,futb_txn_002,Mar22,SynthCash,None,OpenContract,2021-09-02T14:15:00Z,5,FutBund002,EUR,99.50,497500,SynthCashFuturesPortWithDiffCostBasis


We then run a valuation to see the intraday PV of both contracts which is computed as follows:
##### PV = (99.50 \* 10 \* 100,000) / 100 + (99.50 \* 5 \* 100,000) / 100 =  1,492,500 EUR  
Where 99.50 is the latest market price (as well as the transaction price of the latest purchase).

We also see that the synthetic cash amount for both contracts has decreased by the same purchase amount:
##### Change in Synthetic Cash = (99.50 \* 5 \* 100,000) / 100 = 497,500

In [33]:
get_daily_fut_val("2021-09-02T14:15:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,492,500.00","1,492,500.00","100,000.00",99.50,15.00,"1,497,500.00"
4,Mar22,Mar22Future,"1,492,500.00","1,492,500.00","100,000.00",99.50,15.00,"1,502,500.00"
3,Mar22,PnLCash,"5,000.00","5,000.00",1.00,NaN,"5,000.00","5,000.00"
1,Dec21,SynthCash,"-1,497,500.00","-1,497,500.00",1.00,NaN,"-1,497,500.00","-1,497,500.00"
2,Mar22,SynthCash,"-1,502,500.00","-1,502,500.00",1.00,NaN,"-1,502,500.00","-1,502,500.00"


In [34]:
get_daily_fut_val("2021-09-02T14:15:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"-5,000.00","-5,000.00",None,99.50,None,None
1,Mar22,"-5,000.00","-5,000.00",None,99.50,None,None


#### End of Day
At the close of the day, both Futures prices have moved down further to 98.00. Our PVs are now computed as follows:

##### PV = (98.00 \* 10 \* 100,000) / 100 + (98.00 \* 5 \* 100,000) / 100 =  1,470,500 EUR  

In [35]:
get_daily_fut_val("2021-09-02T17:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,470,000.00","1,470,000.00","100,000.00",98.00,15.00,"1,497,500.00"
4,Mar22,Mar22Future,"1,470,000.00","1,470,000.00","100,000.00",98.00,15.00,"1,502,500.00"
3,Mar22,PnLCash,"5,000.00","5,000.00",1.00,NaN,"5,000.00","5,000.00"
1,Dec21,SynthCash,"-1,497,500.00","-1,497,500.00",1.00,NaN,"-1,497,500.00","-1,497,500.00"
2,Mar22,SynthCash,"-1,502,500.00","-1,502,500.00",1.00,NaN,"-1,502,500.00","-1,502,500.00"


In [36]:
get_daily_fut_val("2021-09-02T17:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"-27,500.00","-27,500.00",None,98.00,None,None
1,Mar22,"-27,500.00","-27,500.00",None,98.00,None,None


## Day 3
#### Start of Day
At 8:00am on Day 3, we book a 'RealisePnlDecrease' transaction to adjust the cost basis of the Mar 22 Bund Future downwards in the amount of yesterday's P&L giving us a start of day PV of 0.


In [37]:
futures_transactions.iloc[5:6]

,txn_id,fut_code,fut_synth_cash_code,fut_pnl_cash_code,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
5,rpnbl_txn_002,Mar22,SynthCash,PnLCash,RealisePnLDecrease,2021-09-03T08:00:00Z,0,FutBund002,EUR,0.00,32500,SynthCashFuturesPortWithDiffCostBasis


We then take a start of day valuation of the two contracts and can see the PVs remain unchanged from the previous days close with a market price of 98.00. The PnL cash line has decreased by 32,500 EUR.
At the start of the day, we take a valuation with a price of 98.00.

In [38]:
get_daily_fut_val("2021-09-03T08:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,470,000.00","1,470,000.00","100,000.00",98.00,15.00,"1,497,500.00"
4,Mar22,Mar22Future,"1,470,000.00","1,470,000.00","100,000.00",98.00,15.00,"1,470,000.00"
3,Mar22,PnLCash,"-27,500.00","-27,500.00",1.00,NaN,"-27,500.00","-27,500.00"
1,Dec21,SynthCash,"-1,497,500.00","-1,497,500.00",1.00,NaN,"-1,497,500.00","-1,497,500.00"
2,Mar22,SynthCash,"-1,470,000.00","-1,470,000.00",1.00,NaN,"-1,470,000.00","-1,470,000.00"


In [39]:
get_daily_fut_val("2021-09-03T08:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"-27,500.00","-27,500.00",None,98.00,None,None
1,Mar22,"-27,500.00","-27,500.00",None,98.00,None,None


#### Intraday

Intraday, we take a valuation of our portfolio where the price has moved to 104.00

##### PV = (104.00 * 15 * 10,000) / 100 = 1,560,000 EUR


In [40]:
get_daily_fut_val("2021-09-03T14:15:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,560,000.00","1,560,000.00","100,000.00",104.00,15.00,"1,497,500.00"
4,Mar22,Mar22Future,"1,560,000.00","1,560,000.00","100,000.00",104.00,15.00,"1,470,000.00"
3,Mar22,PnLCash,"-27,500.00","-27,500.00",1.00,NaN,"-27,500.00","-27,500.00"
1,Dec21,SynthCash,"-1,497,500.00","-1,497,500.00",1.00,NaN,"-1,497,500.00","-1,497,500.00"
2,Mar22,SynthCash,"-1,470,000.00","-1,470,000.00",1.00,NaN,"-1,470,000.00","-1,470,000.00"


In [41]:
get_daily_fut_val("2021-09-03T14:15:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"62,500.00","62,500.00",None,104.00,None,None
1,Mar22,"62,500.00","62,500.00",None,104.00,None,None


#### End of Day

Similar to our intraday valuation, we see the price hasn't changed at 104.00. We take a final end of day valuation.

##### PV = (104.00 * 15 * 10,000) / 100 = 1,560,000 EUR

In [42]:
get_daily_fut_val("2021-09-03T17:00:00Z", portfolio_code, False)

,FutCode,FutHoldType,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,Dec21Future,"1,560,000.00","1,560,000.00","100,000.00",104.00,15.00,"1,497,500.00"
4,Mar22,Mar22Future,"1,560,000.00","1,560,000.00","100,000.00",104.00,15.00,"1,470,000.00"
3,Mar22,PnLCash,"-27,500.00","-27,500.00",1.00,NaN,"-27,500.00","-27,500.00"
1,Dec21,SynthCash,"-1,497,500.00","-1,497,500.00",1.00,NaN,"-1,497,500.00","-1,497,500.00"
2,Mar22,SynthCash,"-1,470,000.00","-1,470,000.00",1.00,NaN,"-1,470,000.00","-1,470,000.00"


In [43]:
get_daily_fut_val("2021-09-03T17:00:00Z", portfolio_code, True)

,FutCode,PV,Exposure,Contract Size,Price,Units,Cost
0,Dec21,"62,500.00","62,500.00",None,104.00,None,None
1,Mar22,"62,500.00","62,500.00",None,104.00,None,None
